In [9]:
import pretty_midi
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

In [82]:
def note_vec(note):
    return np.array([note.start, note.end, note.pitch, note.velocity])

In [87]:
for i in tqdm(range(1, 910)):
    idx = f"{i:0>3}"
    mid = pretty_midi.PrettyMIDI(f"POP909/POP909/{idx}/{idx}.mid")
    for inst in mid.instruments:
        seq = pd.DataFrame(np.vstack([note_vec(note) for note in inst.notes]), columns = ['start','end','pitch','velocity'])
        seq.to_csv(rf"POP909/POP909/{idx}/{inst.name}.txt")
        

100%|█████████████████████████████████████████| 909/909 [01:25<00:00, 10.61it/s]


In [109]:
for i in tqdm(range(1, 910)):
    idx = f"{i:0>3}"
    beat = pd.read_csv(rf"POP909/POP909/{idx}/beat_midi.txt", sep=" ", header = None) 
    beat = beat.rename(columns = {0: 'start', 1: 'beat1', 2: 'beat2'})
    mel = pd.read_csv(rf"POP909/POP909/{idx}/MELODY.txt", sep=",") 
    mel_merge = pd.merge(beat, mel, how= 'outer', left_on = 'start', right_on = 'start', suffixes=('_beat', '_melody'))

100%|████████████████████████████████████████| 909/909 [00:02<00:00, 361.36it/s]


In [34]:
i = 1
idx = f"{i:0>3}"
beat = pd.read_csv(rf"POP909/POP909/{idx}/beat_midi.txt", sep=" ", header = None) 
beat = beat.rename(columns = {0: 'start', 1: 'beat1', 2: 'beat2'})
mel = pd.read_csv(rf"POP909/POP909/{idx}/MELODY.txt", sep=",", index_col = 0) 

mel['duration'] = mel.end - mel.start

In [47]:
# try to match the beat and notes if possible
mat = pd.merge_asof(mel, beat, on = 'start', suffixes=('_mel', '_beat'), direction='nearest', tolerance = 1e-3)
check = pd.merge_asof(beat, mel, on = 'start', suffixes=('_beat', '_mel'), direction='nearest', tolerance = 1e-3)
print(check)
mat = mat._append(check[np.isnan(check.end)]).sort_values(by = 'start')
print(mat)

          start  beat1  beat2  end  pitch  velocity  duration
0      0.055333    1.0    1.0  NaN    NaN       NaN       NaN
1      0.721998    0.0    0.0  NaN    NaN       NaN       NaN
2      1.388663    1.0    0.0  NaN    NaN       NaN       NaN
3      2.055328    0.0    0.0  NaN    NaN       NaN       NaN
4      2.721993    1.0    1.0  NaN    NaN       NaN       NaN
..          ...    ...    ...  ...    ...       ...       ...
287  191.388188    0.0    0.0  NaN    NaN       NaN       NaN
288  192.054853    1.0    1.0  NaN    NaN       NaN       NaN
289  192.721518    0.0    0.0  NaN    NaN       NaN       NaN
290  193.388183    1.0    0.0  NaN    NaN       NaN       NaN
291  194.054848    0.0    0.0  NaN    NaN       NaN       NaN

[292 rows x 7 columns]
          start  end  pitch  velocity  duration  beat1  beat2
0      0.055333  NaN    NaN       NaN       NaN    1.0    1.0
1      0.721998  NaN    NaN       NaN       NaN    0.0    0.0
2      1.388663  NaN    NaN       NaN       Na